<a href="https://colab.research.google.com/github/hajsf/Time-Series-Forecast/blob/main/07_PredictFuture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import numpy as np

In [ ]:
# import required library
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

**Read the csv files**


*   Define correct seperator, tab in our example
*   Use dtype to enforce data type mainy in casefield field has mix data types
*   Use low_memory=False so you do not mind whatever type the undefined fields are read in they have mix data





In [ ]:
pathData = "/content/drive/MyDrive/ColabNotebooks/dataset/02_ProcessedData/"
pathExternal = "/content/drive/MyDrive/ColabNotebooks/dataset/03_ExternalData/"

**DATAFRAME creation**

In [ ]:
import hashlib

def hashSKU(s):  
      return int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16) % (10 ** 8)

In [ ]:
SLS = [58, 100]
SLS = [float(i) for i in SLS]

SKUs = [hashSKU("124766")]

df = (pd.DataFrame({'DATE1': pd.date_range('2020-08-01', '2020-08-31')})
     .merge(pd.Series(SLS, name='SLS_CNTR_ID'), how="cross")
     .merge(pd.Series(SKUs, name='hashedSKU'), how="cross"))

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   DATE1        62 non-null     datetime64[ns]
 1   SLS_CNTR_ID  62 non-null     float64       
 2   hashedSKU    62 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.9 KB


**Feature Creation**

1. Hijri calendar

In [ ]:
from hijri_converter import Hijri, Gregorian

def hijriDate(row):  
    y = row['DATE1'].year
    m = row['DATE1'].month
    d = row['DATE1'].day
    return Gregorian(y, m, d).to_hijri()

def isRamadan(dt):  
    if dt.month == 9:
      return 1
    else:
      return 0

def isRamadanEnd(dt):  
    if dt.month == 9 and dt.day >= 21:
      return 1
    else:
      return 0

def isEid(dt):  
    if (dt.month == 10 and dt.day < 4) or (dt.month == 12 and dt.day < 9 and dt.day > 14):
      return 1
    else:
      return 0

2. Gregorian calendar

In [ ]:
def isWeekend(dt):  
    if dt.weekday() == 4 or dt.weekday() == 5: # Monday is 0
      return 1
    else:
      return 0

def isSalaryweek(dt):  
    if dt.day >= 25:
      return 1
    else:
      return 0

def isNationalday(dt):  
    if (dt.month == 2 and dt.day == 22) or (dt.month == 9 and dt.day == 23) :
      return 1
    else:
      return 0

3. Assign features related to calendar DATE1

In [ ]:
df['dayofyear'] = df['DATE1'].apply(lambda dt: dt.dayofyear)
df['Monthday'] = df['DATE1'].apply(lambda dt: dt.day)
df['Weekday'] = df['DATE1'].apply(lambda dt: dt.dayofweek)
df['weekofyear'] = df['DATE1'].apply(lambda dt: dt.week)
df['month'] = df['DATE1'].apply(lambda dt: dt.month)
df['quarter'] = df['DATE1'].apply(lambda dt: dt.quarter)

df['isSalaryweek'] = df['DATE1'].apply(lambda dt: isSalaryweek(dt))
df['isWeekend'] = df['DATE1'].apply(lambda dt: isWeekend(dt))
df['isNationalday'] = df['DATE1'].apply(lambda dt: isNationalday(dt))

df['hijri'] = df.apply(lambda row: hijriDate(row), axis=1)
df['isRamadan'] = df['hijri'].apply(lambda dt: isRamadan(dt))
df['isRamadanEnd'] = df['hijri'].apply(lambda dt: isRamadanEnd(dt))
df['isEid'] = df['hijri'].apply(lambda dt: isEid(dt))

df = df.drop(columns=['hijri'])

In [ ]:
calenderFeatures = pd.read_csv(pathExternal +'calenderFeatures.csv', sep=',', lineterminator='\r', low_memory=False)
calenderFeatures = calenderFeatures.replace('\n','', regex=True)
calenderFeatures['DATE1'] = pd.to_datetime(calenderFeatures['DATE1'], format='%Y-%m-%d %H:%M:%S')
df = pd.merge(df, 
              calenderFeatures, 
              on =['DATE1'], 
              how ='inner')

In [ ]:
Categories = pd.read_csv(pathExternal +'Categories.csv', sep=',', lineterminator='\r', low_memory=False)
Categories = Categories.replace('\n','', regex=True)
Categories[['CL_1']] = Categories[['CL_1']].astype(float, errors = 'raise')
Categories = Categories[['CL_1','Category']]
Categories = Categories.reset_index()
Categories = Categories.drop(columns=['index'])
Categories = Categories.dropna(subset=['Category']) 
Categories.dtypes

CL_1        float64
Category     object
dtype: object

In [ ]:
SKUcategory = pd.read_csv(pathExternal +'ITEMS_MASTER.csv', sep='\t', lineterminator='\r', low_memory=False)
SKUcategory = SKUcategory.replace('\n','', regex=True)
SKUcategory[['ITEM_ID']] = SKUcategory[['ITEM_ID']].astype(str, errors = 'raise')
SKUcategory = SKUcategory.reset_index()
SKUcategory = SKUcategory.drop(columns=['index'])
SKUcategory = pd.merge(SKUcategory, 
              Categories, 
              on =['CL_1'], 
              how ='inner')
SKUcategory = SKUcategory[['ITEM_ID','Category']]
SKUcategory.dtypes

ITEM_ID     object
Category    object
dtype: object

In [ ]:
oneHot = (pd.get_dummies(SKUcategory.Category, prefix='Category').reset_index())
SKUcategory = SKUcategory.join(oneHot)
SKUcategory = SKUcategory.drop(columns=['Category', 'index'])

In [ ]:
SKUcategory['hashedSKU'] = SKUcategory['ITEM_ID'].apply(lambda s: hashSKU(s))

In [ ]:
SKUcategory = SKUcategory.drop(columns=['ITEM_ID'])

In [ ]:
df = pd.merge(df, 
              SKUcategory, 
              on =['hashedSKU'], 
              how ='inner')

In [ ]:
FEATURES = list(df.columns.values)
print('number of features:', len(FEATURES))
FEATURES

**Load Model**

In [ ]:
import pickle
with open(pathData + 'model.pk', 'rb') as f:
  model = pickle.load(f)
print("model:", model)

model: XGBRegressor(base_score=0.05, early_stopping_rounds=50, learning_rate=0.001,
             max_depth=10, missing=nan, n_estimators=1000,
             objective='reg:squarederror', random_state=5)


**Predict Future**

In [ ]:
df = df.drop(columns=['ForecastedQty'])

In [ ]:
df.dtypes

In [ ]:
df.set_index("DATE1", inplace = True)

number of features: 28


['SLS_CNTR_ID',
 'hashedSKU',
 'dayofyear',
 'Monthday',
 'Weekday',
 'weekofyear',
 'month',
 'quarter',
 'isSalaryweek',
 'isWeekend',
 'isNationalday',
 'isRamadan',
 'isRamadanEnd',
 'isEid',
 'Holiday',
 'AcademicCalendarSemester',
 'Category_Brand',
 'Category_Brand2',
 'Category_Cancelled',
 'Category_HomeMade',
 'Category_HomeMadewithVAT',
 'Category_Imported',
 'Category_Itproducts',
 'Category_Local',
 'Category_Packaging',
 'Category_Partner',
 'Category_RawMaterial',
 'Category_SisterCompany']

In [ ]:
df['ForecastedQty'] = model.predict(df)

ValueError: ignored

**Draw Chart**

In [ ]:
#data = data.reset_index()

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(25, 5))
plt.plot(data.index.get_level_values('DATE1'), data['ForecastedQty'])
plt.ylabel('Forecast Qty')

plt.ylim([5, 15]) #ymax is your value
# or as below (same applicable for xlim):
#plt.ylim(bottom=5) #ymin is your value
#plt.ylim(top=7) #ymax is your value
print('Toal forecasted quantity =', data['ForecastedQty'].sum())

plt.title("Daily Forecast")
plt.show()